## Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re

In [ ]:
# Load dataset

df = pd.read_csv("train_preprocess.tsv.txt" ,encoding="latin1",sep='\t',header=None,names=["text","label"])

In [ ]:
df.head()

,text,label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


In [ ]:
df.shape

(11000, 2)

In [ ]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
#Cek duplikasi data

df.duplicated().sum()

67

In [ ]:
# Hapus duplicated data

df = df.drop_duplicates()

In [ ]:
print('Jumlah Duplikasi Data = {}'.format(df.duplicated().sum()))
print('Duplikasi Data Tela dihapus...')

Jumlah Duplikasi Data = 0
Duplikasi Data Tela dihapus...


In [ ]:
#cek lagi
df.shape

(10933, 2)

In [ ]:
df.head()

,text,label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


- Ada 67 data yang diduplikasi dalam dataset dan sudah dihapus, Update terdiri dari 10933 baris.

## Text Processing

In [ ]:
df_alay = pd.read_csv('new_kamusalay.csv', encoding='ISO-8859-1', header=None)
df_alay = df_alay.rename(columns={0: 'alay', 1: 'formal'})
df_alay_dict = dict(zip(df_alay['alay'], df_alay['formal']))

In [ ]:
def case_folding (text):
    return text.lower()

def clean (text):
    clean1 = re.sub ('\\n','', text)
    clean2 = re.sub ('RT',' ', clean1)
    clean3 = re.sub ('USER', ' ', clean2)
    clean4 = re.sub ('(http|https):\/\/s+', ' ', clean3)
    clean5 = re.sub ('[^0-9a-zA-Z]+', ' ', clean4)
    clean6 = re.sub ('x[a-z0-9]{2}', ' ', clean5)
    clean7 = re.sub ("\d+", ' ', clean6)
    clean8 = re.sub ('  +', '', clean7)
    clean9 = re.sub ('user', ' ', clean8)
    return clean9

def tokenization(text):
    text = re.split('\W+', text)
    return text

def alay_normalization(text):
    newlist = []
    for word in text:
        if word in df_alay_dict:
            text = df_alay_dict[word]
            newlist.append(text)
        else:
            text = word
            newlist.append(text)
    return newlist

#stopwords
stopword_list = ["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                 'yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia',
                  'ia', 'seperti', 'jika', 'sehingga', 'kembali', 'dan', 'ini', 'karena', 'kepada', 'oleh',
                  'saat', 'sementara', 'setelah', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah',
                  'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bahwa', 'atau', 'kita', 'dengan',
                  'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda',
                  'kalo', 'amp', 'biar', 'bikin', 'bilang', 'si', 'tau', 'tdk', 'tuh', 'utk', 'ya','kayak', 'i', 'a',
                  'gak', 'ga', 'krn', 'nya', 'nih', 'sih','jd', 'jgn', 'sdh', 'aja', 'n', 't', 'gue', 'yah',
                 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang',
                  'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun',
                  'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'seolah', 'seraya',
                  'seterusnya', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'mari', 'nanti', 'oh', 'ok',
                  'setiap', 'sesuatu','saja', 'toh', 'walau', 'amat', 'apalagi', 'dengan', 'bahwa', 'oleh']

stopword_list = set(stopword_list)

def remove_stopwords(text):
    text = [word for word in text if word not in stopword_list]
    return text

def clean_non_existed(text):
    if text == '':
        return None
    else:
        return text

#jadikan satu fungsi cleansing
def cleansing(text):
    text = case_folding(text)
    text = clean(text)
    text = tokenization(text)
    text = alay_normalization(text)
    text = remove_stopwords(text)
    text = ' '.join(text)
    text = clean_non_existed(text)
    return text

In [ ]:
# Apply cleansing
df['text'] = df['text'].apply(cleansing)

In [ ]:
df.head()

,text,label
0,warung dimiliki pengusaha pabrik tahu puluhan ...,positive
1,mohon ulama lurus k mmbri hujjah pak ai apa ha...,neutral
2,lokasi strategis jalan sumatra bandung tempat ...,positive
3,betapa bahagia diri unboxing paket barang bagu...,positive
4,aduh jadi mahasiswa jangan sombong dong kasih ...,negative


In [ ]:
import sqlite3

# Membuat koneksi ke database SQLite
conn = sqlite3.connect('database.db')

In [ ]:
# Menyimpan DataFrame ke dalam tabel 'tabel_cleansed' dalam database
df.to_sql('tabel_cleansed', con=conn, index=False, if_exists='replace')

10933

In [ ]:
print("DataFrame yang telah di-cleansing berhasil disimpan ke dalam tabel 'tabel_cleansed' di database.")

DataFrame yang telah di-cleansing berhasil disimpan ke dalam tabel 'tabel_cleansed' di database.


In [ ]:
# Close the database

conn.commit()
conn.close()